In [1]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Ruta del proyecto en Google Drive
PROJECT_DIR = "/content/drive/MyDrive/TFG-FakeNewsNet"

# Me muevo a la carpeta principal del proyecto
%cd $PROJECT_DIR

!ls

/content/drive/MyDrive/TFG-FakeNewsNet
data  models  notebooks  README.md  requirements.txt  results


In [3]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer # Para TF-IDF
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import save_npz
import joblib

import os
import re


In [4]:
# Cargo el dataset generado en el Notebook 02

df = pd.read_csv("data/noticias_preproc.csv")

print("Shape:", df.shape)
display(df.head(3))
display(df.info())


Shape: (44246, 15)


,title,text,subject,date,label,text_clean,n_chars,n_words,avg_word_len,n_exclam,n_question,n_digits,n_upper_words,url_count,has_url
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1,21st century wire say ben stein reputable prof...,1028,171,6.011696,0,0,7,12,0,0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0,washington reuter president donald trump remov...,4820,771,6.251621,0,0,6,19,0,0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0,reuter puerto rico governor ricardo rossello s...,1848,304,6.078947,0,0,0,4,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44246 entries, 0 to 44245
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          44246 non-null  object 
 1   text           44246 non-null  object 
 2   subject        44246 non-null  object 
 3   date           44246 non-null  object 
 4   label          44246 non-null  int64  
 5   text_clean     44161 non-null  object 
 6   n_chars        44246 non-null  int64  
 7   n_words        44246 non-null  int64  
 8   avg_word_len   44246 non-null  float64
 9   n_exclam       44246 non-null  int64  
 10  n_question     44246 non-null  int64  
 11  n_digits       44246 non-null  int64  
 12  n_upper_words  44246 non-null  int64  
 13  url_count      44246 non-null  int64  
 14  has_url        44246 non-null  int64  
dtypes: float64(1), int64(9), object(5)
memory usage: 5.1+ MB


None

#Verificar "text_clean"

In [10]:
# Compruebo si existe la columna
assert "text_clean" in df.columns, "ERROR: falta la columna text_clean"

# Identifico filas con valores nulos (NaN)
n_missing = df["text_clean"].isna().sum()
print("Filas con NaN:", n_missing)

Filas con NaN: 85


Al comprobar que hay valores nulos, los regenero con la función de "limpieza y lematización" usadas en el notebook 02

In [11]:
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm  # barra de progreso en operaciones lentas
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Activo las barras de progreso en pandas
tqdm.pandas()

# Stopwords en inglés
STOPWORDS = set(ENGLISH_STOP_WORDS)

# Patrón para detectar URLs
URL_PATTERN = re.compile(r"http\S+|www\.\S+")

# Patrón para tokenizar palabras (deja solo letras de a-z y números simples de 0-9)
TOKEN_PATTERN = re.compile(r"\b[a-z0-9]+\b")


In [12]:
if n_missing > 0:
    print("Regenerando text_clean para las filas afectadas...")

    def simple_lemma(token: str) -> str:   # Lematizar muy  ligero

        t = token

        #  Pasar sufijos -ies → -y
        if len(t) > 4 and t.endswith("ies"):
            return t[:-3] + "y"

        # Eliminar sufijos "ing" y "ed"
        for suffix in ["ing", "ed"]:
            if len(t) > 4 and t.endswith(suffix):
                t = t[:-len(suffix)]
                break

        # Quitar plurales (-s, -es)
        if len(t) > 3 and t.endswith("es"):
            t = t[:-2]
        elif len(t) > 2 and t.endswith("s"):
            t = t[:-1]

        return t

    def clean_and_normalize(text: str) -> str:  # Limpieza necesaria en los modelos

        text = text.lower()  # Convierte el texto en minúsculas
        text = URL_PATTERN.sub(" ", text) # Reemplazar las URLs por un espacio

        tokens = TOKEN_PATTERN.findall(text) # Tokenización simple (quita alfanuméricos)

        clean_tokens = []
        for tok in tokens:
            if len(tok) <= 2:   # Eliminar palabras cortas
                continue
            if tok in STOPWORDS:    # Eliminar stopwords sin significado
                continue
            clean_tokens.append(simple_lemma(tok))   # Aplicar la función de arriba

        return " ".join(clean_tokens)   # Devuelve ya el texto limpio


    # Regenerar solo las filas con NaN
    mask_missing = df["text_clean"].isna()
    df.loc[mask_missing, "text_clean"] = df.loc[mask_missing, "text"].progress_apply(clean_and_normalize)

    print("Regeneración completada.")

Regenerando text_clean para las filas afectadas...


  0%|          | 0/85 [00:00<?, ?it/s]

Regeneración completada.


In [13]:
# Verificación final
assert df["text_clean"].isna().sum() == 0, "ERROR: text_clean contiene valores nulos tras la regeneración"

print("Verificación final realizada correctamente.")
df["text_clean"].head()

Verificación final realizada correctamente.


,text_clean
0,21st century wire say ben stein reputable prof...
1,washington reuter president donald trump remov...
2,reuter puerto rico governor ricardo rossello s...
3,monday donald trump embarras country accidenta...
4,glasgow scotland reuter presidential candidat ...


Ejecuto de nuevo esta celda para comprobar si ya hay valores nulos o no

In [14]:
# Identifico filas con valores nulos (NaN)
n_missing = df["text_clean"].isna().sum()
print("Filas con NaN:", n_missing)

Filas con NaN: 0


# Creación columna "domain"

In [15]:
assert "subject" in df.columns, "ERROR: falta la columna 'subject'."

df["domain"] = df["subject"]

print("Columna 'domain' creada correctamente.")
df[["domain", "title"]].head()

Columna 'domain' creada correctamente.


,domain,title
0,US_News,Ben Stein Calls Out 9th Circuit Court: Committ...
1,politicsNews,Trump drops Steve Bannon from National Securit...
2,politicsNews,Puerto Rico expects U.S. to lift Jones Act shi...
3,News,OOPS: Trump Just Accidentally Confirmed He Le...
4,politicsNews,Donald Trump heads for Scotland to reopen a go...


#Creación vector TF-IDF

In [16]:
tfidf_vectorizer = TfidfVectorizer(
    max_features= 50000,  # Limito el tamaño máx. para que sea estable
    min_df= 3, # Busco que una palabra aparezca mín. en 3 documentos
    max_df= 0.7,  # Si la palabra aparece más del 70% en noticias, produce mucho ruido
    ngram_range= (1, 2),  # Incluir unigramas y bigramas
)

print(tfidf_vectorizer)

TfidfVectorizer(max_df=0.7, max_features=50000, min_df=3, ngram_range=(1, 2))


# Generar matriz TF-IDF

In [17]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(df["text_clean"])

print("Dimensiones:", tfidf_matrix.shape)
print("Densidad:", tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]))

Dimensiones: (44246, 50000)
Densidad: 0.003840731365547168
CPU times: user 40.1 s, sys: 1.14 s, total: 41.2 s
Wall time: 42.9 s


# Agrupación noticias por dominio

Transformo la matriz TF-IDF a nivel de noticia en una matriz TF-IDF a nivel de dominio.

In [18]:
assert "domain" in df.columns, "ERROR: falta la columna 'domain'"   # Compruebo que se generó la columna.


In [19]:
df_domains = df[["domain"]].copy()
tfidf_dom_matrix = []    # Aquí se almacenan los vectores TF-IDF de cada dominio

domains_unique = df["domain"].unique()

for dom in domains_unique:
    idx = (df["domain"] == dom).values    # Creo una máscara booleana para seleccionar las noticias del dominio "dom"
    tfidf_dom_matrix.append(tfidf_matrix[idx].mean(axis=0))  # Selecciono las filas de la matriz que pertenecen a ese dominio

tfidf_dom_matrix = np.vstack(tfidf_dom_matrix)   # La lista se convierte en una matriz NumPy

print("Vector por dominio generado correctamente.")
print("Shape:", tfidf_dom_matrix.shape)

Vector por dominio generado correctamente.
Shape: (8, 50000)


# Creación matriz de similitud coseno de dominios

Mide cuánto se parecen semánticamente los dominios en función del contenido que publican (es la base del grafo de contenido).

In [22]:
tfidf_dom_matrix = np.asarray(tfidf_dom_matrix)   # Convierte la matriz de vectores por dominio en array NumPy
sim_matrix = cosine_similarity(tfidf_dom_matrix)  # Calcula la matriz  de similitud coseno entre dominios

print("Matriz de similitud generada:", sim_matrix.shape)

Matriz de similitud generada: (8, 8)


# Generación aristas del grafo

Generar conexiones entre dominios cuando sim_matrix es suficientemente alta

In [23]:
edges = []

thr = 0.10  # Umbral de similitud mínima

for i, dom_i in enumerate(domains_unique):        # Recorrer todos los pares posibles de dominios
    for j, dom_j in enumerate(domains_unique):
        if j <= i:         # Evita duplicados y que se compare un dominio consigo mismo
            continue
        sim = sim_matrix[i, j]
        if sim >= thr:            # Descarta las similitudes que no sean relevantes
            edges.append((dom_i, dom_j, float(sim)))

edges_df = pd.DataFrame(edges, columns=["domain_A", "domain_B", "similarity"])

print("Total de aristas:", len(edges_df))
edges_df.head(10)

Total de aristas: 28


,domain_A,domain_B,similarity
0,US_News,politicsNews,0.459242
1,US_News,News,0.473685
2,US_News,Government News,0.530405
3,US_News,left-news,0.532447
4,US_News,worldnews,0.419654
5,US_News,politics,0.542288
6,US_News,Middle-east,0.999924
7,politicsNews,News,0.710353
8,politicsNews,Government News,0.741333
9,politicsNews,left-news,0.689353


# Guardado de los resultados

In [24]:
# Creo carpetas si no existen
os.makedirs("models", exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("results", exist_ok=True)

joblib.dump(tfidf_vectorizer, "models/tfidf_vectorizer.joblib")    # Guarda tfidf_vectorizer ya entrenado

# Guardado de la matriz TF-IDF
save_npz("data/tfidf_matrix.npz", tfidf_matrix, compressed=False)

# Guardado de las aristas de grafo
edges_df.to_csv("results/content_edges_tfidf.csv", index=False)

print("Vector TF-IDF guardado en models/")
print("Matriz TF-IDF guardada sin compresión en data/")
print("Aristas guardadas en results/content_edges_tfidf.csv")


Vector TF-IDF guardado en models/
Matriz TF-IDF guardada sin compresión en data/
Aristas guardadas en results/content_edges_tfidf.csv
